In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("openai:gpt-4o-mini", temperature=0)

In [ ]:
prompt = """
You are an assistant that helps users with warehouse management.
You have:
- 110 😷 masks
- 20 📦 boxes
- 10 📦 boxes
"""

messages=[
    ("system", prompt),
    ("user", "How many boxes do I have?")
]

response = llm.invoke(messages)
response.pretty_print()

================================== Ai Message ==================================

You have a total of 30 📦 boxes (20 + 10).


In [46]:
from langchain_core.tools import tool
import requests

@tool("get_warehouse_inventory", description="Get the current inventory of the warehouse")
def get_warehouse_inventory():
    """Get the current inventory of the warehouse"""
    return f"masks: {110}, boxes: {30}"

@tool("add_mask", description="Add a number of masks to the warehouse")
def add_mask(count: int):
    """Add a number of masks to the warehouse"""
    return f"masks: {110 + count}"

@tool("add_box", description="Add a number of boxes to the warehouse")
def add_box(count: int):
    """Add a number of boxes to the warehouse"""
    return f"boxes: {30 + count}"

@tool("busca_productos_platzi") #Nombre personalizado para la tool
def search_products() -> str:
  """Busca productos en la API de Platzi"""
  # Conexión a API
  url = "https://api.escuelajs.co/api/v1/products?offset=0&limit=1"
  response = requests.get(url)
  products = response.json()
  return "".join([f"{product['title']} - ${product['price']}\n" for product in products])

@tool("get_city_cordinates", description="Get coordinates for a given city")
def get_city_cordinates(city: str):
    """Return coordinates by city"""
    import requests
    from urllib.parse import quote

    url = f"https://geocoding-api.open-meteo.com/v1/search?name={quote(city)}&count=1&language=es&format=json"
    response = requests.get(url)
    data = response.json()

    if "results" not in data or not data["results"]:
        return []

    result = data["results"][0]
    latitude = result["latitude"]
    longitude = result["longitude"]

    return [latitude, longitude]

@tool("check_weather", description="Get current weather for a specified location (city name or coordinates)")
def check_weather(latitude="-40.157", longitude="-71.353"):
    print("latitude", latitude)
    "Get weather for a specified location"
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true")
    weather = response.json()

    return weather

tools = [get_city_cordinates, check_weather]

In [47]:
[latitude, longitude] = get_city_cordinates.invoke("medellin")
print(latitude, longitude)

6.25184 -75.56359


In [50]:
check_weather.invoke({
    "latitude": latitude,
    "longitude": longitude
})

latitude 6.25184


{'latitude': 6.25,
 'longitude': -75.5,
 'generationtime_ms': 0.07081031799316406,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 1493.0,
 'current_weather_units': {'time': 'iso8601',
  'interval': 'seconds',
  'temperature': '°C',
  'windspeed': 'km/h',
  'winddirection': '°',
  'is_day': '',
  'weathercode': 'wmo code'},
 'current_weather': {'time': '2026-01-18T03:45',
  'interval': 900,
  'temperature': 18.7,
  'windspeed': 8.0,
  'winddirection': 100,
  'is_day': 0,
  'weathercode': 2}}

In [34]:
prompt = """
    You are an assistant that helps users with warehouse management and weather information.

    You have the following tools:
    - get_weather_by_city: Get the weather information for a given city
    - check_weather: Get current weather for a specified location (city name or coordinates)

    You can use the tools to help the user with their request.
    If you don't have the information to answer the user's question, just say "I don't know"
"""

messages=[
    ("system", prompt),
    ("user", "could you tell me the weather in medellin tomorrow")
]

ll_with_tools = llm.bind_tools(tools)
response = ll_with_tools.invoke(messages)
response.tool_calls

[]

image.png